In [1]:
import pandas as pd
import json
import numpy as np
import datetime
from pymongo import MongoClient
import Orange
import matplotlib.pyplot as plt
from jupyter_dash import JupyterDash
import plotly.graph_objects as go
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import talib

# Create Mongo client instance
mongo_client=MongoClient()

In [13]:
db=mongo_client['Market5']
data=db['ACC'].find()
eqdf=pd.DataFrame(list(data))
eqdf.drop('_id',inplace=True,axis=1)
eqdf.drop_duplicates(inplace=True)

In [15]:
eqdf['Datetime']=pd.to_datetime(eqdf['Datetime']).dt.tz_localize('UTC').dt.tz_convert('Asia/Kolkata')
eqdf.set_index('Datetime',inplace=True)


In [ ]:
dfcopy=eqdf.copy()

In [92]:
def a_d(df):    
    df['mfm']=(2*(df['Close'])-df['Low']-df['High'])/(df['High']-df['Low'])
    df['mfv']=df['mfm']*df['Volume']
    df['a/d']=df['mfm'].cumsum()
    df['a/d_v']=df['mfv'].cumsum()
    df.replace(to_replace=np.nan,value=0,inplace=True)
    return df

In [93]:
df=a_d(dfcopy)

In [108]:
df['a/d_ma']=df['a/d'].rolling(26).mean()
df.replace(to_replace=np.nan,value=0,inplace=True)

In [98]:
df

,Open,High,Low,Close,Volume,mfm,mfv,a/d,a/d_v,a/d_ma
Datetime,,,,,,,,,,
2021-09-17 09:15:00+05:30,2441.199951,2441.250000,2425.899902,2431.100098,570.0,-0.322454,-183.799027,-0.322454,-183.799027,0.000000
2021-09-17 09:20:00+05:30,2433.000000,2439.949951,2431.449951,2436.000000,5575.0,0.070600,393.593463,-0.251855,209.794436,0.000000
2021-09-17 09:25:00+05:30,2435.899902,2437.250000,2427.000000,2428.250000,3415.0,-0.756098,-2582.073171,-1.007952,-2372.278735,0.000000
2021-09-17 09:30:00+05:30,2427.000000,2428.000000,2423.050049,2426.649902,2373.0,0.454501,1078.529963,-0.553452,-1293.748772,0.000000
2021-09-17 09:35:00+05:30,2428.149902,2428.149902,2422.000000,2422.050049,3858.0,-0.983724,-3795.206034,-1.537175,-5088.954806,0.000000
...,...,...,...,...,...,...,...,...,...,...
2021-12-22 14:10:00+05:30,2142.149902,2144.449951,2141.149902,2141.399902,3069.0,-0.848487,-2604.006880,-108.220636,24858.129563,-106.366498
2021-12-22 14:15:00+05:30,2141.399902,2145.550049,2140.850098,2144.699951,3694.0,0.638253,2357.704950,-107.582384,27215.834513,-106.651286
2021-12-22 14:20:00+05:30,2144.550049,2145.750000,2143.000000,2145.750000,1751.0,1.000000,1751.000000,-106.582384,28966.834513,-106.829922


In [102]:
### This whole section to plot charts in Dash
def genrate_graphs():
    app = JupyterDash(__name__)
    app.layout = html.Div(children=[
            html.H1(children='Hello Dash'),

            html.Div(children='''
                Dash: A web application framework for your data.
            '''),
            dcc.Checklist(
                id='toggle-rangeslider',
                options=[{'label': 'Include Rangeslider', 
                          'value': 'slider'}],
                value=['slider']
            ),
            dcc.Graph(
                id='graph',
                # figure=fig
            )
        ])
    @app.callback(Output("graph", "figure"), [Input("toggle-rangeslider", "value")])
    def show_chart(value):
        fig=make_subplots(rows=3,cols=1,column_widths=[1.0],
                                  row_heights=[1.0,0.75,0.75],shared_xaxes=True,vertical_spacing=0.01,
                                  specs=[
                                      [{"type":"xy","secondary_y": True}],#None],
                                      # [{"type":"Candlestick"}],#None],                              
                                      # [{"type":"xy","secondary_y": True}],#None],
                                      [{"type":"xy"}],#None]])
                                      [{"type":"xy"}],#None]]
                                        ]
                                 )
        fig.add_trace(
                    go.Candlestick(name='price',x=df.index,
                                open=df['Open'],
                                high=df['High'],
                                low=df['Low'],
                                close=df['Close']),row=1,col=1,secondary_y=False)
        fig.update_layout(xaxis_rangeslider_visible=False)
 
        fig.add_trace(go.Scatter(x=df.index,y=df['a/d'], name="A/D",mode='lines'),row=2,col=1)
        fig.add_trace(go.Scatter(x=df.index,y=df['a/d_ma'], name="13 ma",mode='lines'),row=2,col=1)
        fig.add_trace(go.Scatter(x=df.index,y=df['a/d_v'], name="A/D_V",mode='lines'),row=3,col=1)
        fig.update_xaxes(row=1,col=1,
                         rangeselector=dict(buttons=list(
                             [dict(count=1, label="1m", step="month", stepmode="backward"),
                              dict(count=6, label="6m", step="month", stepmode="backward"),
                              dict(count=1, label="YTD", step="year", stepmode="todate"),
                              dict(count=1, label="1y", step="year", stepmode="backward"),
                              dict(step="all")
                             ])
                                           ),
                         rangebreaks=[
                             dict(bounds=["sat", "mon"]),#hide weekends
                             dict(bounds=[15.5, 9.25], pattern="hour")
                         ]
                        )
                # rangebreaks=[
            #     dict(bounds=[15, 9], pattern="hour"), #hide hours outside of 9am-5pm
            # ]
        fig.update_layout(autosize=False,width=1200,height=800,
                          margin=dict(l=50,r=50,b=100,t=100,pad=4),
                          paper_bgcolor="LightSteelBlue",)
        return fig
        
    app.run_server(mode='jupyterlab',port = 8097)

In [109]:
genrate_graphs()